## 1. Setup

In [51]:
import sys
import os
import h5py
import gc
from glob import glob
from pathlib import Path
from functools import lru_cache
from collections import defaultdict
from fastprogress import progress_bar
from copy import deepcopy
from tqdm import tqdm

import numpy as np
import pandas as pd
from PIL import Image, ExifTags
import cv2
import sqlite3

from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import DBSCAN

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import backend as K
import torch
import kornia
from kornia.feature import (
    LoFTR, LocalFeature, PassLAF, LAFOrienter, PatchDominantGradientOrientation, OriNet, LAFAffNetShapeEstimator,
    KeyNetDetector, LAFDescriptor, HardNet8, HyNet, TFeat, SOSNet, get_laf_center, DescriptorMatcher
)
from transformers import TFViTModel, ViTFeatureExtractor

import pycolmap

In [42]:
root = Path('/home/gunes/Desktop/Kaggle/image-matching-challenge-2023')
competition_dataset = root / 'data'

In [43]:
sys.path.append(str(root / 'venv' / 'lib' / 'python3.9' / 'site-packages' / 'SuperGluePretrainedNetwork'))
from models.matching import Matching

In [46]:
df = pd.read_csv(competition_dataset / 'sample_submission.csv')

if df.shape[0] != 3:
    # Enable submission mode and disable verbose
    submission = True
    verbose = False
    train_or_test_directory = competition_dataset / 'test'
else:
    # Disable submission mode and enable verbose
    submission = False
    verbose = True
    train_or_test_directory = competition_dataset / 'train'

if submission is False:
    df = pd.read_csv(competition_dataset / 'train' / 'train_labels.csv')
    
print(f'Dataset Shape: {df.shape}')

Dataset Shape: (327, 5)


In [49]:
def load_angle_detection_model():
    
    """
    Load angle detection model and processor
    
    Returns
    -------
    model: keras.engine.functional.Functional
        Angle detection model
        
    processor: transformers.ViTFeatureExtractor
        Image processor
    """
    
    vit = TFViTModel.from_pretrained(root / 'models' / 'vit-base-patch16-224')
    
    input_layer = Input(shape=(3, 224, 224))
    x = vit(input_layer)
    y = Dense(1, activation='linear')(x[-1])
    
    model = Model(input_layer, y)
    model.load_weights(root / 'models' / 'deep-image-orientation-angle-detection' / 'model-vit-ang-loss.h5')
    
    processor = ViTFeatureExtractor.from_pretrained(root / 'models' / 'vit-base-patch16-224')
    
    return model, processor


def detect_angle(image, model, processor):
    
    """
    Detect angle of a given image
    
    Parameters
    ----------
    image: numpy.ndarray of shape (height, width, channel)
        Image array
        
    model: keras.engine.functional.Functional
        Angle detection model
        
    processor: transformers.ViTFeatureExtractor
        Image processor
    
    Returns
    -------
    outputs: float
    """
    
    inputs = processor(images=[image], return_tensors='np')['pixel_values']
    outputs = model.predict(inputs, verbose=None)[0][0]
        
    return outputs


In [52]:
angle_detection_model, angle_detection_processor = load_angle_detection_model()

for idx, row in tqdm(df.iterrows(), total=df.shape[0]):

    image_path = train_or_test_directory / row['image_path']
    image = np.array(Image.open(image_path))
    
    # Extract image dimensions and memory usage
    df.loc[idx, 'image_height'] = image.shape[0]
    df.loc[idx, 'image_width'] = image.shape[1]
    df.loc[idx, 'memory_usage'] = image.nbytes
    
    # Extract image orientation
    angle = detect_angle(
        image=image,
        model=angle_detection_model,
        processor=angle_detection_processor
    )
    df.loc[idx, 'angle'] = -angle
    

df['image_id'] = df['image_path'].apply(lambda x: str(x).split('/')[-1])
df['memory_usage'] /= (1024 ** 2)

del angle_detection_model, angle_detection_processor
gc.collect()
torch.cuda.empty_cache()
K.clear_session()

Some layers from the model checkpoint at /home/gunes/Desktop/Kaggle/image-matching-challenge-2023/models/vit-base-patch16-224 were not used when initializing TFViTModel: ['classifier']
- This IS expected if you are initializing TFViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFViTModel were not initialized from the model checkpoint at /home/gunes/Desktop/Kaggle/image-matching-challenge-2023/models/vit-base-patch16-224 and are newly initialized: ['vit/pooler/dense/kernel:0', 'vit/pooler/dense/bias:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%

In [53]:
df

,dataset,scene,image_path,rotation_matrix,translation_vector,image_height,image_width,memory_usage,image_id,angle
0,urban,kyiv-puppet-theater,urban/kyiv-puppet-theater/images/IMG_20220127_...,0.961211667939937;0.012901284585988954;0.27550...,-15.272578207792487;-0.6299638684926413;21.960...,1600.0,1200.0,5.493164,IMG_20220127_170709.jpg,-0.264195
1,urban,kyiv-puppet-theater,urban/kyiv-puppet-theater/images/IMG_20220127_...,0.9983992256415107;0.03067020011152836;0.04752...,-10.86744827847772;5.142649233329267;22.080480...,1600.0,1200.0,5.493164,IMG_20220127_170633.jpg,-1.230195
2,urban,kyiv-puppet-theater,urban/kyiv-puppet-theater/images/IMG_20220127_...,-0.00887553412819142;-0.176226804404349;-0.984...,21.65171058540772;5.995022279588175;8.02005533...,1200.0,1200.0,4.119873,IMG_20220127_170350.jpg,5.902641
3,urban,kyiv-puppet-theater,urban/kyiv-puppet-theater/images/IMG_20220127_...,-0.9161929266043187;-0.08177524286162957;-0.39...,9.821364270813396;4.839277289238847;19.6308283...,1200.0,1200.0,4.119873,IMG_20220127_170339.jpg,2.854750
4,urban,kyiv-puppet-theater,urban/kyiv-puppet-theater/images/IMG_20220127_...,0.9917061795507187;0.02380210836276639;0.12630...,-25.98610607878878;1.9810845701842204;-1.04975...,1200.0,1200.0,4.119873,IMG_20220127_170250.jpg,-21.354853
...,...,...,...,...,...,...,...,...,...,...
322,heritage,wall,heritage/wall/images/DSC_5051_acr.jpg,0.9996327491725756;-0.023470477496970843;0.013...,-24.878330444800003;-0.307958687968;0.10009750...,4032.0,6048.0,69.767578,DSC_5051_acr.jpg,-0.084656
323,heritage,wall,heritage/wall/images/DSC_5054_acr.jpg,0.998539039945242;-0.0008816636815971297;0.054...,-26.397828666400002;0.25433805800000003;1.0851...,4032.0,6048.0,69.767578,DSC_5054_acr.jpg,-0.805049
324,heritage,wall,heritage/wall/images/DSC_5057_acr.jpg,0.9981761687264716;-0.008441133163217011;0.059...,-27.9128179912;0.08558456216799999;1.30660573568,4032.0,6048.0,69.767578,DSC_5057_acr.jpg,-1.323586
325,heritage,wall,heritage/wall/images/DSC_5060_acr.jpg,0.9999248231217353;-0.010114127992376822;-0.00...,-29.1611086616;0.126412881824;-0.4765696612,4032.0,6048.0,69.767578,DSC_5060_acr.jpg,-0.437576


## 2. Image Utilities

In [ ]:
@lru_cache(maxsize=32)
def read_image(image_path):
    return cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)


def create_image_pairs(image_paths):

    """
    Create all possible image pairs from given list of image paths

    Parameters
    ----------
    image_paths: list of shape (n_images)
        List of image paths

    Returns
    -------
    image_pair_indices: list of shape (n_image_pairs)
        List of tuples of image pair indices
    """

    image_pair_indices = []

    for i in range(len(image_paths)):
        for j in range(i + 1, len(image_paths)):
            image_pair_indices.append((i, j))

    return image_pair_indices


def resize_with_aspect_ratio(image, longest_edge):

    """
    Resize image while preserving its aspect ratio

    Parameters
    ----------
    image: numpy.ndarray of shape (height, width, 3)
        Image array

    longest_edge: int
        Desired number of pixels on the longest edge

    Returns
    -------
    image: numpy.ndarray of shape (resized_height, resized_width, 3)
        Resized image array
    """

    height, width = image.shape[:2]
    scale = longest_edge / max(height, width)
    image = cv2.resize(image, dsize=(int(width * scale), int(height * scale)), interpolation=cv2.INTER_LANCZOS4)

    return image


def get_image_tensor(image_path_or_array, resize, resize_shape, resize_longest_edge, scale, grayscale):

    """
    Load image and preprocess it

    Parameters
    ----------
    image_path_or_array: str or numpy.ndarray of shape (height, width, 3)
        Image path or image array

    resize: bool
        Whether to resize the image or not

    resize_shape: tuple or int
        Tuple of image height and width or number of pixels for both height and width

    resize_longest_edge: bool
        Whether to resize the longest edge or not

    scale: bool
        Whether to scale image pixel values by max 8-bit pixel value or not

    grayscale: bool
        Whether to convert RGB image to grayscale or not

    Returns
    -------
    image: torch.Tensor of shape (1, 1 or 3, height, width)
        Image tensor
    """

    if isinstance(image_path_or_array, pathlib.Path) or isinstance(image_path_or_array, str):
        # Read image from the given path if image_path_or_array is a path-like string
        image = cv2.imread(str(image_path_or_array))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    else:
        image = image_path_or_array

    if resize:
        if resize_longest_edge:
            image = resize_with_aspect_ratio(image=image, longest_edge=resize_shape)
        else:
            resize_shape = (resize_shape, resize_shape) if isinstance(resize_shape, int) else resize_shape
            image = cv2.resize(image, resize_shape, interpolation=cv2.INTER_LANCZOS4)

    if scale:
        image = image / 255.

    image = kornia.image_to_tensor(image, False).float()
    if grayscale:
        image = kornia.color.rgb_to_grayscale(image)

    return image


def crop(image, keypoints, perc_points=0.85, pad=5):
    
    norm_keypoints = MinMaxScaler().fit_transform(keypoints)
    total = len(keypoints)
    best_dist = 1
    best_clusters = None
    best_asm = None
    
    for eps in [0.01, 0.025, 0.05, 0.1, 0.2]:
        clusters = DBSCAN(eps=eps).fit_predict(norm_keypoints)
        counts = pd.Series(clusters).value_counts().sort_values(ascending=False)
        counts = counts[counts.index > -1]
        
        if len(counts) == 0:
            continue

        cumsums = np.cumsum(counts.values) / total
        dists = np.abs(cumsums - perc_points)
        best_ix = np.argmin(dists)

        if dists[best_ix] < best_dist:
            best_dist = dists[best_ix]
            best_clusters = list(counts.head(best_ix + 1).index)
            best_asm = clusters

    mask = np.isin(best_asm, best_clusters)

    miny = int(np.min(keypoints[mask][:, 1]))
    miny = max(miny - pad, 0)

    maxy = int(np.max(keypoints[mask][:, 1]))
    maxy = min(maxy + pad, image.shape[0])

    minx = int(np.min(keypoints[mask][:, 0]))
    minx = max(minx - pad, 0)

    maxx = int(np.max(keypoints[mask][:, 0]))
    maxx = min(maxx + pad, image.shape[1])

    image_cropped = image[miny:maxy + 1, minx:maxx + 1, :]
    
    keypoints_cropped = np.copy(keypoints)
    keypoints_cropped[:, 0] -= minx
    keypoints_cropped[:, 1] -= miny
    keypoints_cropped = keypoints_cropped[
        (keypoints_cropped[:, 0] > 0) &
        (keypoints_cropped[:, 1] > 0) &
        (keypoints_cropped[:, 0] < image_cropped.shape[1]) &
        (keypoints_cropped[:, 1] < image_cropped.shape[0])
    ]

    return image_cropped, keypoints_cropped, minx, miny


## 3. Camera Utilities

In [ ]:
def get_focal_length(image_path):

    """
    Get focal length from EXIF or calculate it using prior

    Parameters
    ----------
    image_path: str
        Image path

    Returns
    -------
    focal_length: float
        Focal length extracted from EXIF or calculated using prior
    """

    image = Image.open(image_path)
    image_longest_edge = max(image.size)

    focal_length = None
    exif = image.getexif()

    if exif is not None:

        focal_length_35mm = None

        for tag, value in exif.items():
            if ExifTags.TAGS.get(tag, None) == 'FocalLengthIn35mmFilm':
                focal_length_35mm = float(value)

        if focal_length_35mm is not None:
            focal_length = focal_length_35mm / 35. * image_longest_edge

    if focal_length is None:
        prior_focal_length = 1.2
        focal_length = prior_focal_length * image_longest_edge

    return focal_length


## 4. Database Utilities

In [ ]:
MAX_IMAGE_ID = 2**31 - 1

CREATE_CAMERAS_TABLE = """CREATE TABLE IF NOT EXISTS cameras (
    camera_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    model INTEGER NOT NULL,
    width INTEGER NOT NULL,
    height INTEGER NOT NULL,
    params BLOB,
    prior_focal_length INTEGER NOT NULL)"""

CREATE_DESCRIPTORS_TABLE = """CREATE TABLE IF NOT EXISTS descriptors (
    image_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    FOREIGN KEY(image_id) REFERENCES images(image_id) ON DELETE CASCADE)"""

CREATE_IMAGES_TABLE = """CREATE TABLE IF NOT EXISTS images (
    image_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    name TEXT NOT NULL UNIQUE,
    camera_id INTEGER NOT NULL,
    prior_qw REAL,
    prior_qx REAL,
    prior_qy REAL,
    prior_qz REAL,
    prior_tx REAL,
    prior_ty REAL,
    prior_tz REAL,
    CONSTRAINT image_id_check CHECK(image_id >= 0 and image_id < {}),
    FOREIGN KEY(camera_id) REFERENCES cameras(camera_id))
""".format(MAX_IMAGE_ID)

CREATE_TWO_VIEW_GEOMETRIES_TABLE = """
CREATE TABLE IF NOT EXISTS two_view_geometries (
    pair_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    config INTEGER NOT NULL,
    F BLOB,
    E BLOB,
    H BLOB)
"""

CREATE_KEYPOINTS_TABLE = """CREATE TABLE IF NOT EXISTS keypoints (
    image_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    FOREIGN KEY(image_id) REFERENCES images(image_id) ON DELETE CASCADE)
"""

CREATE_MATCHES_TABLE = """CREATE TABLE IF NOT EXISTS matches (
    pair_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB)"""

CREATE_NAME_INDEX = \
    "CREATE UNIQUE INDEX IF NOT EXISTS index_name ON images(name)"

CREATE_ALL = "; ".join([
    CREATE_CAMERAS_TABLE,
    CREATE_IMAGES_TABLE,
    CREATE_KEYPOINTS_TABLE,
    CREATE_DESCRIPTORS_TABLE,
    CREATE_MATCHES_TABLE,
    CREATE_TWO_VIEW_GEOMETRIES_TABLE,
    CREATE_NAME_INDEX
])


def image_ids_to_pair_id(image_id1, image_id2):
    if image_id1 > image_id2:
        image_id1, image_id2 = image_id2, image_id1
    return image_id1 * MAX_IMAGE_ID + image_id2


def pair_id_to_image_ids(pair_id):
    image_id2 = pair_id % MAX_IMAGE_ID
    image_id1 = (pair_id - image_id2) / MAX_IMAGE_ID
    return image_id1, image_id2


def array_to_blob(array):
    return array.tobytes()


def blob_to_array(blob, dtype, shape=(-1,)):
    return np.fromstring(blob, dtype=dtype).reshape(*shape)


class COLMAPDatabase(sqlite3.Connection):

    @staticmethod
    def connect(database_path):
        return sqlite3.connect(database_path, factory=COLMAPDatabase)


    def __init__(self, *args, **kwargs):
        super(COLMAPDatabase, self).__init__(*args, **kwargs)

        self.create_tables = lambda: self.executescript(CREATE_ALL)
        self.create_cameras_table = \
            lambda: self.executescript(CREATE_CAMERAS_TABLE)
        self.create_descriptors_table = \
            lambda: self.executescript(CREATE_DESCRIPTORS_TABLE)
        self.create_images_table = \
            lambda: self.executescript(CREATE_IMAGES_TABLE)
        self.create_two_view_geometries_table = \
            lambda: self.executescript(CREATE_TWO_VIEW_GEOMETRIES_TABLE)
        self.create_keypoints_table = \
            lambda: self.executescript(CREATE_KEYPOINTS_TABLE)
        self.create_matches_table = \
            lambda: self.executescript(CREATE_MATCHES_TABLE)
        self.create_name_index = lambda: self.executescript(CREATE_NAME_INDEX)

    def add_camera(self, model, width, height, params,
                   prior_focal_length=False, camera_id=None):
        params = np.asarray(params, np.float64)
        cursor = self.execute(
            "INSERT INTO cameras VALUES (?, ?, ?, ?, ?, ?)",
            (camera_id, model, width, height, array_to_blob(params),
             prior_focal_length))
        return cursor.lastrowid

    def add_image(self, name, camera_id,
                  prior_q=np.zeros(4), prior_t=np.zeros(3), image_id=None):
        cursor = self.execute(
            "INSERT INTO images VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
            (image_id, name, camera_id, prior_q[0], prior_q[1], prior_q[2],
             prior_q[3], prior_t[0], prior_t[1], prior_t[2]))
        return cursor.lastrowid

    def add_keypoints(self, image_id, keypoints):
        assert(len(keypoints.shape) == 2)
        assert(keypoints.shape[1] in [2, 4, 6])

        keypoints = np.asarray(keypoints, np.float32)
        self.execute(
            "INSERT INTO keypoints VALUES (?, ?, ?, ?)",
            (image_id,) + keypoints.shape + (array_to_blob(keypoints),))

    def add_descriptors(self, image_id, descriptors):
        descriptors = np.ascontiguousarray(descriptors, np.uint8)
        self.execute(
            "INSERT INTO descriptors VALUES (?, ?, ?, ?)",
            (image_id,) + descriptors.shape + (array_to_blob(descriptors),))

    def add_matches(self, image_id1, image_id2, matches):
        assert(len(matches.shape) == 2)
        assert(matches.shape[1] == 2)

        if image_id1 > image_id2:
            matches = matches[:,::-1]

        pair_id = image_ids_to_pair_id(image_id1, image_id2)
        matches = np.asarray(matches, np.uint32)
        self.execute(
            "INSERT INTO matches VALUES (?, ?, ?, ?)",
            (pair_id,) + matches.shape + (array_to_blob(matches),))

    def add_two_view_geometry(self, image_id1, image_id2, matches,
                              F=np.eye(3), E=np.eye(3), H=np.eye(3), config=2):
        assert(len(matches.shape) == 2)
        assert(matches.shape[1] == 2)

        if image_id1 > image_id2:
            matches = matches[:,::-1]

        pair_id = image_ids_to_pair_id(image_id1, image_id2)
        matches = np.asarray(matches, np.uint32)
        F = np.asarray(F, dtype=np.float64)
        E = np.asarray(E, dtype=np.float64)
        H = np.asarray(H, dtype=np.float64)
        self.execute(
            "INSERT INTO two_view_geometries VALUES (?, ?, ?, ?, ?, ?, ?, ?)",
            (pair_id,) + matches.shape + (array_to_blob(matches), config,
             array_to_blob(F), array_to_blob(E), array_to_blob(H)))

In [ ]:
def create_camera(db, image_path, camera_model):
    
    image = Image.open(image_path)
    width, height = image.size

    focal_length = get_focal_length(image_path)

    if camera_model == 'simple-pinhole':
        model = 0
        param_arr = np.array([focal_length, width / 2, height / 2])
    if camera_model == 'pinhole':
        model = 1
        param_arr = np.array([focal_length, focal, width / 2, height / 2])
    elif camera_model == 'simple-radial':
        model = 2
        param_arr = np.array([focal_length, width / 2, height / 2, 0.1])
    elif camera_model == 'opencv':
        model = 4
        param_arr = np.array([focal_length, focal_length, width / 2, height / 2, 0., 0., 0., 0.])
         
    return db.add_camera(model, width, height, param_arr)


def add_keypoints(db, h5_path, image_path, camera_model, single_camera=True):
    
    keypoint_f = h5py.File(os.path.join(h5_path, 'keypoints.h5'), 'r')
    camera_id = None
    fname_to_id = {}
    
    for filename in tqdm(list(keypoint_f.keys())):
        
        keypoints = keypoint_f[filename][()]

        path = os.path.join(image_path, filename)
        if not os.path.isfile(path):
            raise IOError(f'Invalid image path {path}')

        if camera_id is None or not single_camera:
            camera_id = create_camera(db, path, camera_model)
        image_id = db.add_image(filename, camera_id)
        fname_to_id[filename] = image_id

        db.add_keypoints(image_id, keypoints)

    return fname_to_id


def add_matches(db, h5_path, fname_to_id):
    
    match_file = h5py.File(os.path.join(h5_path, 'matches.h5'), 'r')
    
    added = set()
    n_keys = len(match_file.keys())
    n_total = (n_keys * (n_keys - 1)) // 2

    with tqdm(total=n_total) as pbar:
        for key_1 in match_file.keys():
            group = match_file[key_1]
            for key_2 in group.keys():
                id_1 = fname_to_id[key_1]
                id_2 = fname_to_id[key_2]

                pair_id = image_ids_to_pair_id(id_1, id_2)
                if pair_id in added:
                    continue
            
                matches = group[key_2][()]
                db.add_matches(id_1, id_2, matches)
                added.add(pair_id)
                pbar.update(1)
                
                
def get_unique_idxs(A, dim=0):
    
    unique, idx, counts = torch.unique(A, dim=dim, sorted=True, return_inverse=True, return_counts=True)
    _, ind_sorted = torch.sort(idx, stable=True)
    cum_sum = counts.cumsum(0)
    cum_sum = torch.cat((torch.tensor([0],device=cum_sum.device), cum_sum[:-1]))
    first_indices = ind_sorted[cum_sum]
    
    return first_indices

## 5. Evaluation

In [ ]:
def array_to_string(a):

    """
    Flatten given array and convert it to a string with semicolon delimiters

    Parameters
    ----------
    a: np.ndarray
        N-dimensional array

    Returns
    -------
    s: string
        String form of the given array
    """

    s = ';'.join([str(x) for x in a.reshape(-1)])

    return s


def string_to_array(s):

    """
    Convert semicolon delimited string to an array

    Parameters
    ----------
    s: string
        String form of the array

    Returns
    -------
    a: np.ndarray
        N-dimensional array
    """

    a = np.array(s.split(';')).astype(np.float64)

    return a


def rotation_matrix_to_quaternion(rotation_matrix):

    """
    Convert rotation matrix to quaternion

    Parameters
    ----------
    rotation_matrix: numpy.ndarray of shape (3, 3)
        Array of directions of the world-axes in camera coordinates

    Returns
    -------
    quaternion: numpy.ndarray of shape (4)
        Array of quaternion
    """

    r00 = rotation_matrix[0, 0]
    r01 = rotation_matrix[0, 1]
    r02 = rotation_matrix[0, 2]
    r10 = rotation_matrix[1, 0]
    r11 = rotation_matrix[1, 1]
    r12 = rotation_matrix[1, 2]
    r20 = rotation_matrix[2, 0]
    r21 = rotation_matrix[2, 1]
    r22 = rotation_matrix[2, 2]

    k = np.array([
        [r00 - r11 - r22, 0.0, 0.0, 0.0],
        [r01 + r10, r11 - r00 - r22, 0.0, 0.0],
        [r02 + r20, r12 + r21, r22 - r00 - r11, 0.0],
        [r21 - r12, r02 - r20, r10 - r01, r00 + r11 + r22]
    ])
    k /= 3.0

    # Quaternion is the eigenvector of k that corresponds to the largest eigenvalue
    w, v = np.linalg.eigh(k)
    quaternion = v[[3, 0, 1, 2], np.argmax(w)]

    if quaternion[0] < 0:
        np.negative(quaternion, quaternion)

    return quaternion


def pose_difference(r1, t1, r2, t2):

    """
    Calculate relative pose difference from given rotation matrices and translation vectors

    Parameters
    ----------
    r1: numpy.ndarray of shape (3, 3)
        First rotation matrix

    t1: numpy.ndarray of shape (3)
        First translation vector

    r2: numpy.ndarray of shape (3, 3)
        Second rotation matrix

    t2: numpy.ndarray of shape (3)
        Second translation vector

    Returns
    -------
    rotation_difference: float
        Rotation difference in terms of degrees from the first image

    translation_difference: float
        Translation difference in terms of meters from the first image
    """

    rotation_difference = np.dot(r2, r1.T)
    translation_difference = t2 - np.dot(rotation_difference, t1)

    return rotation_difference, translation_difference


def rotation_and_translation_error(q_ground_truth, t_ground_truth, q_prediction, t_prediction, epsilon=1e-15):

    """
    Calculate rotation and translation error

    Parameters
    ----------
    q_ground_truth: numpy.ndarray of shape (4)
        Array of quaternion derived from ground truth rotation matrix

    t_ground_truth: numpy.ndarray of shape (3)
        Array of ground truth translation vector

    q_prediction: numpy.ndarray of shape (4)
        Array of quaternion derived from estimated rotation matrix

    t_prediction: numpy.ndarray of shape (3)
        Array of estimated translation vector

    epsilon: float
        A small number for preventing zero division

    Returns
    -------
    rotation_error: float
        Rotation error in terms of degrees

    translation_error: float
        Translation error in terms of meters
    """

    q_ground_truth_norm = q_ground_truth / (np.linalg.norm(q_ground_truth) + epsilon)
    q_prediction_norm = q_prediction / (np.linalg.norm(q_prediction) + epsilon)
    loss_q = np.maximum(epsilon, (1.0 - np.sum(q_prediction_norm * q_ground_truth_norm) ** 2))

    rotation_error = np.degrees(np.arccos(1 - (2 * loss_q)))

    scaling_factor = np.linalg.norm(t_ground_truth)
    t_prediction = scaling_factor * (t_prediction / (np.linalg.norm(t_prediction) + epsilon))
    translation_error = min(
        np.linalg.norm(t_ground_truth - t_prediction),
        np.linalg.norm(t_ground_truth + t_prediction)
    )

    return rotation_error, translation_error


def mean_average_accuracy(rotation_errors, translation_errors, rotation_error_thresholds, translation_error_thresholds):

    """
    Calculate mean average accuracies over a set of thresholds for rotation and translation

    Parameters
    ----------
    rotation_errors: list of shape (n_pairs)
        List of rotation errors

    translation_errors: list of shape (n_pairs)
        List of translation errors

    rotation_error_thresholds: numpy.ndarray of shape (10)
        Array of rotation error thresholds

    translation_error_thresholds: numpy.ndarray of shape (10)
        Array of translation error thresholds

    Returns
    -------
    maa: float
        Mean average accuracy calculated on both rotation and translation errors

    rotation_maa: float
        Mean average accuracy calculated on rotation errors

    translation_maa: float
        Mean average accuracy calculated on translation errors
    """

    accuracies, rotation_accuracies, translation_accuracies = [], [], []

    for rotation_error_threshold, translation_error_threshold in zip(rotation_error_thresholds, translation_error_thresholds):

        # Calculate whether the errors are less than specified thresholds or not
        rotation_accuracy = (rotation_errors <= rotation_error_threshold)
        translation_accuracy = (translation_errors <= translation_error_threshold)
        accuracy = rotation_accuracy & translation_accuracy

        accuracies.append(accuracy.astype(np.float32).mean())
        rotation_accuracies.append(rotation_accuracy.astype(np.float32).mean())
        translation_accuracies.append(translation_accuracy.astype(np.float32).mean())

    maa = np.array(accuracies).mean()
    rotation_maa = np.array(rotation_accuracies).mean()
    translation_maa = np.array(translation_accuracies).mean()

    return maa, rotation_maa, translation_maa


def evaluate(df, verbose=False):

    """
    Calculate mean average accuracies over a set of thresholds for rotation and translation

    Parameters
    ----------
    df: pandas.DataFrame
        Dataframe with dataset, scene, rotation_matrix, translation_vector, rotation_matrix_prediction and translation_vector_prediction columns

    verbose: bool
        Whether to print scores or not

    Returns
    -------
    df_scores: pandas.DataFrame
        Dataframe of scores
    """

    rotation_error_thresholds = {
        **{('haiper', scene): np.linspace(1, 10, 10) for scene in ['bike', 'chairs', 'fountain']},
        **{('heritage', scene): np.linspace(1, 10, 10) for scene in ['cyprus', 'dioscuri']},
        **{('heritage', 'wall'): np.linspace(0.2, 10, 10)},
        **{('urban', 'kyiv-puppet-theater'): np.linspace(1, 10, 10)},
    }
    translation_error_thresholds = {
        **{('haiper', scene): np.geomspace(0.05, 0.5, 10) for scene in ['bike', 'chairs', 'fountain']},
        **{('heritage', scene): np.geomspace(0.1, 2, 10) for scene in ['cyprus', 'dioscuri']},
        **{('heritage', 'wall'): np.geomspace(0.05, 1, 10)},
        **{('urban', 'kyiv-puppet-theater'): np.geomspace(0.5, 5, 10)},
    }
    df_scores = pd.DataFrame(columns=['dataset', 'scene', 'image_pairs', 'maa', 'rotation_maa', 'translation_maa'])

    for (dataset, scene), df_scene in tqdm(df.groupby(['dataset', 'scene'])):

        scene_rotation_errors = []
        scene_translation_errors = []

        for i in range(df_scene.shape[0]):
            for j in range(i + 1, df_scene.shape[0]):

                rotation_matrix_difference_ground_truth, translation_vector_difference_ground_truth = pose_difference(
                    r1=string_to_array((df_scene.iloc[i]['rotation_matrix'])).reshape(3, 3),
                    t1=string_to_array((df_scene.iloc[i]['translation_vector'])),
                    r2=string_to_array((df_scene.iloc[j]['rotation_matrix'])).reshape(3, 3),
                    t2=string_to_array((df_scene.iloc[j]['translation_vector'])),
                )
                quaternion_ground_truth = rotation_matrix_to_quaternion(rotation_matrix=rotation_matrix_difference_ground_truth)

                rotation_matrix_difference_prediction, translation_vector_difference_prediction = pose_difference(
                    r1=string_to_array((df_scene.iloc[i]['rotation_matrix_prediction'])).reshape(3, 3),
                    t1=string_to_array((df_scene.iloc[i]['translation_vector_prediction'])),
                    r2=string_to_array((df_scene.iloc[j]['rotation_matrix_prediction'])).reshape(3, 3),
                    t2=string_to_array((df_scene.iloc[j]['translation_vector_prediction'])),
                )
                quaternion_prediction = rotation_matrix_to_quaternion(rotation_matrix=rotation_matrix_difference_prediction)

                rotation_error, translation_error = rotation_and_translation_error(
                    q_ground_truth=quaternion_ground_truth,
                    t_ground_truth=translation_vector_difference_ground_truth,
                    q_prediction=quaternion_prediction,
                    t_prediction=translation_vector_difference_prediction,
                    epsilon=1e-15
                )
                scene_rotation_errors.append(rotation_error)
                scene_translation_errors.append(translation_error)

        scene_maa, scene_rotation_maa, scene_translation_maa = mean_average_accuracy(
            rotation_errors=scene_rotation_errors,
            translation_errors=scene_translation_errors,
            rotation_error_thresholds=rotation_error_thresholds[(dataset, scene)],
            translation_error_thresholds=translation_error_thresholds[(dataset, scene)]
        )

        if verbose:
            settings.logger.info(
                f'''
                Dataset: {dataset} - Scene: {scene}
                Number of image pairs: {len(scene_rotation_errors)}
                mAA: {scene_maa:.6f} - rotation mAA: {scene_rotation_maa:.6f} - translation mAA: {scene_translation_maa:.6f}
                '''
            )

        df_scores = pd.concat((
            df_scores,
            pd.DataFrame(
                data=[[dataset, scene, len(scene_rotation_errors), scene_maa, scene_rotation_maa, scene_translation_maa]],
                columns=['dataset', 'scene', 'image_pairs', 'maa', 'rotation_maa', 'translation_maa']
            )
        ), axis=0)

    return df_scores


## 6. Models

In [ ]:
def loftr_match_images(image1, image2, model, device, amp, transforms, confidence_threshold, top_k):

    """
    Match given two images with each other using LoFTR model

    Parameters
    ----------
    image1: numpy.ndarray of shape (3, height, width)
        Array of first image

    image2: numpy.ndarray of shape (3, height, width)
        Array of second image

    model: torch.nn.Module
        LoFTR Model

    device: torch.device
        Location of the image1, image2 and the model

    amp: bool
        Whether to use auto mixed precision or not

    transforms: dict
        Dictionary of transform parameters

    confidence_threshold: float or int
        Confidence threshold to filter out low confidence matches

    top_k: int
        Number of matches to take

    Returns
    -------
    outputs: dict
        Model outputs
    """

    image1_raw_height, image1_raw_width = image1.shape[:2]
    image1 = get_image_tensor(
        image_path_or_array=image1,
        resize=transforms['resize'],
        resize_shape=transforms['resize_shape'],
        resize_longest_edge=transforms['resize_longest_edge'],
        scale=transforms['scale'],
        grayscale=transforms['grayscale']
    )
    image1 = image1.to(device)
    image1_transformed_height, image1_transformed_width = image1.shape[2:]

    image2_raw_height, image2_raw_width = image2.shape[:2]
    image2 = get_image_tensor(
        image_path_or_array=image2,
        resize=transforms['resize'],
        resize_shape=transforms['resize_shape'],
        resize_longest_edge=transforms['resize_longest_edge'],
        scale=transforms['scale'],
        grayscale=transforms['grayscale']
    )
    image2 = image2.to(device)
    image2_transformed_height, image2_transformed_width = image2.shape[2:]

    with torch.no_grad():
        if amp:
            with torch.autocast(device_type=device.type, dtype=torch.float16):
                outputs = model({'image0': image1, 'image1': image2})
        else:
            outputs = model({'image0': image1, 'image1': image2})

    for k in outputs.keys():
        outputs[k] = outputs[k].detach().cpu().numpy()

    if confidence_threshold is not None:
        if isinstance(confidence_threshold, float):
            # Select matched keypoints with above given confidence threshold
            confidence_mask = outputs['confidence'] >= confidence_threshold
        elif isinstance(confidence_threshold, int):
            # Select keypoints dynamically based on confidence distribution
            confidence_mean, confidence_std = outputs['confidence'].mean(), outputs['confidence'].std()
            confidence_mask = outputs['confidence'] >= (confidence_mean + (confidence_std * confidence_threshold))
        else:
            raise ValueError(f'Invalid confidence_threshold {confidence_threshold}')

        for k in outputs.keys():
            outputs[k] = outputs[k][confidence_mask]

    if top_k is not None:
        # Select top-k keypoints based on their confidences
        sorting_idx = outputs['matching_scores0'].argsort()[-top_k:]
        for k in outputs.keys():
            outputs[k] = outputs[k][sorting_idx]

    outputs['keypoints0'][:, 0] *= image1_raw_width / image1_transformed_width
    outputs['keypoints0'][:, 1] *= image1_raw_height / image1_transformed_height
    outputs['keypoints1'][:, 0] *= image2_raw_width / image2_transformed_width
    outputs['keypoints1'][:, 1] *= image2_raw_height / image2_transformed_height

    return outputs


In [ ]:
def superglue_match_images(image1, image2, model, device, amp, transforms, score_threshold, top_k):

    """
    Match given two images with each other using SuperGlue model

    Parameters
    ----------
    image1: numpy.ndarray of shape (3, height, width)
        Array of first image

    image2: numpy.ndarray of shape (3, height, width)
        Array of second image

    model: torch.nn.Module
        SuperGlue Model

    device: torch.device
        Location of the image1, image2 and the model

    amp: bool
        Whether to use auto mixed precision or not

    transforms: dict
        Dictionary of transform parameters

    score_threshold: float, int or None
        Confidence threshold

    top_k: int or None
        Number of keypoints to take

    Returns
    -------
    outputs: dict
        Model outputs
    """

    image1_raw_height, image1_raw_width = image1.shape[:2]
    image1 = get_image_tensor(
        image_path_or_array=image1,
        resize=transforms['resize'],
        resize_shape=transforms['resize_shape'],
        resize_longest_edge=transforms['resize_longest_edge'],
        scale=transforms['scale'],
        grayscale=transforms['grayscale']
    )
    image1 = image1.to(device)
    image1_transformed_height, image1_transformed_width = image1.shape[2:]

    image2_raw_height, image2_raw_width = image2.shape[:2]
    image2 = get_image_tensor(
        image_path_or_array=image2,
        resize=transforms['resize'],
        resize_shape=transforms['resize_shape'],
        resize_longest_edge=transforms['resize_longest_edge'],
        scale=transforms['scale'],
        grayscale=transforms['grayscale']
    )
    image2 = image2.to(device)
    image2_transformed_height, image2_transformed_width = image2.shape[2:]
    
    with torch.no_grad():
        if amp:
            with torch.autocast(device_type=device.type, dtype=torch.bfloat16):
                outputs = model({'image0': image1, 'image1': image2})
        else:
            outputs = model({'image0': image1, 'image1': image2})

    for k in outputs.keys():
        if k == 'descriptors0' or k == 'descriptors1':
            outputs[k] = outputs[k][0].detach().cpu().numpy().T
        else:
            outputs[k] = outputs[k][0].detach().cpu().numpy()

    matches_mask = outputs['matches0'] > -1

    for k in ['keypoints1', 'scores1', 'descriptors1', 'matches1', 'matching_scores1']:
        outputs[k] = outputs[k][outputs['matches0'][matches_mask]]

    for k in ['keypoints0', 'scores0', 'descriptors0', 'matches0', 'matching_scores0']:
        outputs[k] = outputs[k][matches_mask]

    if score_threshold is not None:
        if isinstance(score_threshold, float):
            # Select matched keypoints with above given score threshold
            score_mask = outputs['matching_scores0'] >= score_threshold
        elif isinstance(score_threshold, int):
            # Select keypoints dynamically based on score distribution
            score_mean, score_std = outputs['matching_scores0'].mean(), outputs['matching_scores0'].std()
            score_mask = outputs['matching_scores0'] >= (score_mean + (score_std * score_threshold))
        else:
            raise ValueError(f'Invalid score_threshold {score_threshold}')

        for k in outputs.keys():
            outputs[k] = outputs[k][score_mask]

    if top_k is not None:
        # Select top-k keypoints based on their scores
        sorting_idx = outputs['matching_scores0'].argsort()[-top_k:]
        for k in outputs.keys():
            outputs[k] = outputs[k][sorting_idx]

    outputs['keypoints0'][:, 0] *= image1_raw_width / image1_transformed_width
    outputs['keypoints0'][:, 1] *= image1_raw_height / image1_transformed_height
    outputs['keypoints1'][:, 0] *= image2_raw_width / image2_transformed_width
    outputs['keypoints1'][:, 1] *= image2_raw_height / image2_transformed_height

    return outputs


In [ ]:
class LocalFeatureDetectorDescriptor(LocalFeature):

    def __init__(
            self,
            orientation_module_name, orientation_module_parameters, orientation_module_weights_path,
            affine_module_name, affine_module_parameters, affine_module_weights_path,
            detector_module_name, detector_module_parameters, detector_module_weights_path,
            descriptor_module_name, descriptor_module_parameters, descriptor_module_weights_path,
    ):

        """
        Module that combines local feature detector and descriptor

        Parameters
        ----------
        orientation_module_name: str
            Name of the orientation module

        orientation_module_parameters: dict
            Parameters of the orientation module

        orientation_module_weights_path: str
            Path of the orientation module weights

        affine_module_name: str
            Name of the affine module

        affine_module_parameters: dict
            Parameters of the affine module

        affine_module_weights_path: str
            Path of the affine module weights

        detector_module_name: str
            Name of the detector module

        detector_module_parameters: dict
            Parameters of the detector module

        detector_module_weights_path: str
            Path of the detector module weights

        descriptor_module_name: str
            Name of the descriptor module

        descriptor_module_parameters: dict
            Parameters of the descriptor module

        descriptor_module_weights_path: str
            Path of the descriptor module weights

        Returns
        -------
        lafs: torch.Tensor of shape (1, n_detections, 2, 3)
            Detected local affine frames

        responses: torch.Tensor of shape (1, n_detections)
            Response function values for corresponding lafs

        descriptors: torch.Tensor of shape (1, n_detections, n_dimensions)
            Local descriptors
        """

        # Instantiate specified orientation module
        if orientation_module_name == 'PassLAF':
            orientation_module = PassLAF()
        elif orientation_module_name == 'OriNet':
            orientation_module = LAFOrienter(**orientation_module_parameters, angle_detector=OriNet(pretrained=(orientation_module_weights_path is None)))
        elif orientation_module_name == 'PatchDominantGradientOrientation':
            orientation_module = LAFOrienter(**orientation_module_parameters, angle_detector=PatchDominantGradientOrientation())
        else:
            orientation_module = None

        # Instantiate specified affine module
        if affine_module_name == 'LAFAffNetShapeEstimator':
            affine_module = LAFAffNetShapeEstimator(**affine_module_parameters, pretrained=(affine_module_weights_path is None))
        else:
            affine_module = None

        # Instantiate specified detector module
        if detector_module_name == 'KeyNetDetector':
            detector_module = KeyNetDetector(**detector_module_parameters, pretrained=(detector_module_weights_path is None), ori_module=orientation_module, aff_module=affine_module)
        else:
            detector_module = None

        # Load pretrained weights for the detector module
        if orientation_module_weights_path is not None:
            detector_module.ori.angle_detector.load_state_dict(torch.load(orientation_module_weights_path)['state_dict'])
        if affine_module_weights_path is not None:
            detector_module.aff.load_state_dict(torch.load(affine_module_weights_path)['state_dict'])
        if detector_module_weights_path is not None:
            detector_module.model.load_state_dict(torch.load(detector_module_weights_path)['state_dict'])

        # Instantiate specified descriptor module
        if descriptor_module_name == 'HardNet8':
            descriptor_module = LAFDescriptor(**descriptor_module_parameters, patch_descriptor_module=HardNet8(pretrained=(descriptor_module_weights_path is None)))
        elif descriptor_module_name == 'HyNet':
            descriptor_module = LAFDescriptor(**descriptor_module_parameters, patch_descriptor_module=HyNet(pretrained=(descriptor_module_weights_path is None)))
        elif descriptor_module_name == 'TFeat':
            descriptor_module = LAFDescriptor(**descriptor_module_parameters, patch_descriptor_module=TFeat(pretrained=(descriptor_module_weights_path is None)))
        elif descriptor_module_name == 'SOSNet':
            descriptor_module = LAFDescriptor(**descriptor_module_parameters, patch_descriptor_module=SOSNet(pretrained=(descriptor_module_weights_path is None)))
        else:
            descriptor_module = None

        # Load pretrained weights for the descriptor module
        if descriptor_module_weights_path is not None:
            descriptor_module.descriptor.load_state_dict(torch.load(descriptor_module_weights_path))

        super().__init__(detector_module, descriptor_module)


def extract_keypoints_and_descriptors(image, model, amp, device):

    """
    Extract local feature descriptors on given image with given model

    Parameters
    ----------
    image: torch.Tensor of shape (1, 1, height, width)
        Image tensor

    model: torch.nn.Module
        Local feature detector and descriptor model

    device: torch.device
        Location of the image1, image2 and the model

    amp: bool
        Whether to use auto mixed precision or not

    Returns
    -------
    lafs: numpy.ndarray of shape (n_detections, 2, 3)
        Detected local affine frames

    responses: numpy.ndarray of shape (n_detections)
        Response function values for corresponding lafs

    descriptors: numpy.ndarray of shape (n_detections, n_dimensions)
        Local descriptors

    keypoints: numpy.ndarray of shape (n_detections, 2)
        Keypoints
    """

    with torch.no_grad():
        if amp:
            with torch.autocast(device_type=device.type, dtype=torch.float16):
                lafs, responses, descriptors = model(image)
        else:
            lafs, responses, descriptors = model(image)

    responses = torch.squeeze(responses, dim=0).detach().cpu().numpy()
    descriptors = torch.squeeze(descriptors, dim=0).detach().cpu()
    keypoints = get_laf_center(lafs)
    keypoints = keypoints.detach().cpu().numpy().reshape(-1, 2)
    lafs = lafs.detach().cpu().numpy()

    return lafs, responses, descriptors, keypoints


def match_descriptors(descriptors1, descriptors2, matcher):

    """
    Match descriptors with nearest neighbor algorithm

    Parameters
    ----------
    descriptors1: torch.Tensor of shape (n_detections, n_dimensions)
        Descriptors from first image

    descriptors2: torch.Tensor of shape (n_detections, n_dimensions):
        Descriptors from second image

    matcher: torch.nn.Module
        Descriptor matcher

    Returns
    -------
    distances: numpy.ndarray of shape (n_matches)
        Distances of matching descriptors

    indexes: numpy.ndarray of shape (n_matches, 2)
        Indexes of matching descriptors
    """

    with torch.no_grad():
        distances, indexes = matcher(descriptors1, descriptors2)

    distances = distances.detach().cpu().numpy().reshape(-1)
    indexes = indexes.detach().cpu().numpy()

    return distances, indexes


def lfdd_match_images(image1, image2, model, matcher, device, amp, transforms, distance_threshold, top_k):

    """
    Match given two images with each other using given model and matcher

    Parameters
    ----------
    image1: numpy.ndarray of shape (3, height, width)
        Array of first image

    image2: numpy.ndarray of shape (3, height, width)
        Array of second image

    model: torch.nn.Module
        Local feature detector and descriptor model

    matcher: torch.nn.Module
        Descriptor matcher

    device: torch.device
        Location of the image1, image2 and the model

    amp: bool
        Whether to use auto mixed precision or not

    transforms: dict
        Dictionary of transform parameters

    distance_threshold: float
        Threshold to filter out keypoints with low distance

    top_k: int
        Number of keypoints to take

    Returns
    -------
    outputs: dict
        Model outputs
    """

    image1_raw_height, image1_raw_width = image1.shape[:2]
    image1 = get_image_tensor(
        image_path_or_array=image1,
        resize=transforms['resize'],
        resize_shape=transforms['resize_shape'],
        resize_longest_edge=transforms['resize_longest_edge'],
        scale=transforms['scale'],
        grayscale=transforms['grayscale']
    )
    image1 = image1.to(device)
    image1_transformed_height, image1_transformed_width = image1.shape[2:]

    image2_raw_height, image2_raw_width = image2.shape[:2]
    image2 = get_image_tensor(
        image_path_or_array=image2,
        resize=transforms['resize'],
        resize_shape=transforms['resize_shape'],
        resize_longest_edge=transforms['resize_longest_edge'],
        scale=transforms['scale'],
        grayscale=transforms['grayscale']
    )
    image2 = image2.to(device)
    image2_transformed_height, image2_transformed_width = image2.shape[2:]

    _, _, descriptors1, keypoints1 = extract_keypoints_and_descriptors(image=image1, model=model, amp=amp, device=device)
    _, _, descriptors2, keypoints2 = extract_keypoints_and_descriptors(image=image2, model=model, amp=amp, device=device)
    distances, indexes = match_descriptors(descriptors1=descriptors1, descriptors2=descriptors2, matcher=matcher)

    outputs = {
        'keypoints0': keypoints1[indexes[:, 0]],
        'keypoints1': keypoints2[indexes[:, 1]],
        'distances': distances
    }

    if distance_threshold is not None:
        if isinstance(distance_threshold, float):
            # Select matched keypoints with above given distance threshold
            distance_mask = outputs['distances'] >= distance_threshold
        elif isinstance(distance_threshold, int):
            # Select keypoints dynamically based on distance distribution
            distance_mean, distance_std = outputs['distances'].mean(), outputs['distances'].std()
            distance_mask = outputs['distances'] >= (distance_mean + (distance_std * distance_threshold))
        else:
            raise ValueError(f'Invalid distance_threshold {distance_threshold}')

        for k in outputs.keys():
            outputs[k] = outputs[k][distance_mask]

    if top_k is not None:
        # Select top-k keypoints based on their distances
        sorting_idx = outputs['distances'].argsort()[-top_k:]
        for k in outputs.keys():
            outputs[k] = outputs[k][sorting_idx]

    outputs['keypoints0'][:, 0] *= image1_raw_width / image1_transformed_width
    outputs['keypoints0'][:, 1] *= image1_raw_height / image1_transformed_height
    outputs['keypoints1'][:, 0] *= image2_raw_width / image2_transformed_width
    outputs['keypoints1'][:, 1] *= image2_raw_height / image2_transformed_height

    return outputs


In [ ]:
def sift_match_images(image1, image2, model, matcher, transforms, distance_threshold, top_k):

    image1_raw_height, image1_raw_width = image1.shape[:2]
    if transforms['resize']:
        image1 = resize_with_aspect_ratio(
            image=image1,
            longest_edge=transforms['resize_shape']
        )
    image1_transformed_height, image1_transformed_width = image1.shape[:2]

    image2_raw_height, image2_raw_width = image2.shape[:2]
    if transforms['resize']:
        image2 = resize_with_aspect_ratio(
            image=image2,
            longest_edge=transforms['resize_shape']
        )
    image2_transformed_height, image2_transformed_width = image2.shape[:2]
        
    keypoints1, descriptors1 = sift.detectAndCompute(image1, None)
    keypoints2, descriptors2 = sift.detectAndCompute(image2, None)
    matches = matcher.match(descriptors1, descriptors2)
    matches = sorted(matches, key=lambda x: x.distance)
    
    outputs = {
        'keypoints0': np.array([keypoints1[match.queryIdx].pt for match in matches]),
        'keypoints1': np.array([keypoints2[match.trainIdx].pt for match in matches]),
        'distances': np.array([match.distance for match in matches]),
    }
    
    if distance_threshold is not None:
        if isinstance(distance_threshold, float):
            # Select matched keypoints with above given distance threshold
            distance_mask = outputs['distances'] <= distance_threshold
        elif isinstance(distance_threshold, int):
            # Select keypoints dynamically based on distance distribution
            distance_mean, distance_std = outputs['distances'].mean(), outputs['distances'].std()
            distance_mask = outputs['distances'] <= (distance_mean + (distance_std * distance_threshold))
        else:
            raise ValueError(f'Invalid distance_threshold {distance_threshold}')

        for k in outputs.keys():
            outputs[k] = outputs[k][distance_mask]

    if top_k is not None:
        # Select top-k keypoints based on their distances
        sorting_idx = outputs['distances'].argsort()
        for k in outputs.keys():
            outputs[k] = outputs[k][sorting_idx]
    
    outputs['keypoints0'][:, 0] *= image1_raw_width / image1_transformed_width
    outputs['keypoints0'][:, 1] *= image1_raw_height / image1_transformed_height
    outputs['keypoints1'][:, 0] *= image2_raw_width / image2_transformed_width
    outputs['keypoints1'][:, 1] *= image2_raw_height / image2_transformed_height
    
    return outputs


## 7. Inference

In [ ]:
image_matching_device = torch.device('cuda')

# Load LoFTR model with specified configurations
loftr_model = LoFTR(
    pretrained=None,
    config={
        'backbone_type': 'ResNetFPN',
        'resolution': (8, 2),
        'fine_window_size': 5,
        'fine_concat_coarse_feat': True,
        'resnetfpn': {
            'initial_dim': 128,
            'block_dims': [128, 196, 256]
        },
        'coarse': {
            'd_model': 256,
            'd_ffn': 256,
            'nhead': 8,
            'layer_names': ['self', 'cross', 'self', 'cross', 'self', 'cross', 'self', 'cross'],
            'attention': 'linear',
            'temp_bug_fix': False,
        },
        'match_coarse': {
            'thr': 0.2,
            'border_rm': 2,
            'match_type': 'dual_softmax',
            'dsmax_temperature': 0.1,
            'skh_iters': 3,
            'skh_init_bin_score': 1.0,
            'skh_prefilter': True,
            'train_coarse_percent': 0.4,
            'train_pad_num_gt_min': 200,
        },
        'fine': {
            'd_model':128,
            'd_ffn': 128,
            'nhead': 8,
            'layer_names': ['self', 'cross'],
            'attention': 'linear'
        }
    }
)
loftr_model.load_state_dict(torch.load(root / 'models' / 'loftr' / 'loftr_outdoor.ckpt')['state_dict'], strict=False)
loftr_model = loftr_model.eval().to(image_matching_device)

# Load SuperPoint and SuperGlue model with specified configurations
superglue_model = Matching(config={
    'superpoint': {
        'descriptor_dim': 256,
        'nms_radius': 4,
        'keypoint_threshold': 0.01,
        'max_keypoints': -1,
        'remove_borders': 4
    },
    'superglue': {
        'descriptor_dim': 256,
        'weights': 'outdoor',
        'keypoint_encoder': [32, 64, 128, 256],
        'sinkhorn_iterations': 100,
        'match_threshold': 0.2
    }
})
superglue_model = superglue_model.eval().to(image_matching_device)

# Load LFDD model with specified configurations
lfdd_model = LocalFeatureDetectorDescriptor(
    orientation_module_name='OriNet',
    orientation_module_parameters={},
    orientation_module_weights_path=None,
    affine_module_name='LAFAffNetShapeEstimator',
    affine_module_parameters={
        'preserve_orientation': True
    },
    affine_module_weights_path=str(root / 'models' / 'lfdd' / 'affnet.pth'),
    detector_module_name='KeyNetDetector',
    detector_module_parameters={
        'num_features': 2048
    },
    detector_module_weights_path=str(root / 'models' / 'lfdd' / 'keynet.pth'),
    descriptor_module_name='HardNet8',
    descriptor_module_parameters={},
    descriptor_module_weights_path=str(root / 'models' / 'lfdd' / 'hardnet8v2.pt')
)
lfdd_model = lfdd_model.eval().to(image_matching_device)
descriptor_matcher = DescriptorMatcher(match_mode='snn', th=0.9)

# Load SIFT extractor
sift = cv2.SIFT_create()
bf_matcher = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

In [ ]:
def match(read_image_function, image_paths, image_pair_indices, feature_dir, loftr_model, superglue_model):
    
    with h5py.File(f'{feature_dir}/matches_loftr.h5', mode='w') as f_match:
        for pair_idx in progress_bar(image_pair_indices):
            
            idx1, idx2 = pair_idx
            fname1, fname2 = image_paths[idx1], image_paths[idx2]
            key1, key2 = fname1.split('/')[-1], fname2.split('/')[-1]
            
            image1 = read_image_function(fname1)
            image2 = read_image_function(fname2)
            
            sift_outputs = sift_match_images(
                image1=image1,
                image2=image2,
                model=sift,
                matcher=bf_matcher,
                transforms={
                    'resize': True,
                    'resize_shape': 2000,
                    'resize_longest_edge': True,
                    'scale': False,
                    'grayscale': True,
                },
                distance_threshold=None,
                top_k=None
            )
            
            image_pair_longest_edge = np.concatenate([image1.shape[:2], image2.shape[:2]]).max()
            if image_pair_longest_edge > 2000:
                superglue_transforms = {
                    'resize': True,
                    'resize_shape': 1920,
                    'resize_longest_edge': True,
                    'scale': True,
                    'grayscale': True,
                }
            else:
                superglue_transforms = {
                    'resize': False,
                    'resize_shape': None,
                    'resize_longest_edge': None,
                    'scale': True,
                    'grayscale': True,
                }

            
            '''
            loftr_outputs = loftr_match_images(
                image1=image1,
                image2=image2,
                model=loftr_model,
                device=torch.device('cuda'),
                amp=False,
                transforms={
                    'resize': True,
                    'resize_shape': 840,
                    'resize_longest_edge': True,
                    'scale': True,
                    'grayscale': True,
                },
                confidence_threshold=None,
                top_k=None
            )
            '''
            '''
            lfdd_outputs = lfdd_match_images(
                image1=image1,
                image2=image2,
                model=lfdd_model,
                matcher=descriptor_matcher,
                device=torch.device('cuda'),
                amp=False,
                transforms={
                    'resize': True,
                    'resize_shape': 840,
                    'resize_longest_edge': True,
                    'scale': True,
                    'grayscale': True,
                },
                distance_threshold=None,
                top_k=None
            )
            '''
            '''
            superglue_outputs = superglue_match_images(
                image1=image1,
                image2=image2,
                model=superglue_model,
                device=image_matching_device,
                amp=True,
                transforms=superglue_transforms,
                score_threshold=None,
                top_k=None
            )
            '''
            stage2 = False
            if stage2:
                if superglue_outputs['keypoints0'].shape[0] > 2:
                    image1_cropped, keypoints1_cropped, x_offset1, y_offset1 = crop(image1, superglue_outputs['keypoints0'])
                    image2_cropped, keypoints2_cropped, x_offset2, y_offset2 = crop(image2, superglue_outputs['keypoints1'])

                    image_pair_longest_edge = np.concatenate([image1_cropped.shape[:2], image2_cropped.shape[:2]]).max()
                    if image_pair_longest_edge > 2000:
                        superglue_transforms = {
                            'resize': True,
                            'resize_shape': 1920,
                            'resize_longest_edge': True,
                            'scale': True,
                            'grayscale': True,
                        }
                    else:
                        superglue_transforms = {
                            'resize': False,
                            'resize_shape': None,
                            'resize_longest_edge': None,
                            'scale': True,
                            'grayscale': True,
                        }


                    second_stage_superglue_outputs = superglue_match_images(
                        image1=image1_cropped,
                        image2=image2_cropped,
                        model=superglue_model,
                        device=image_matching_device,
                        amp=True,
                        transforms=superglue_transforms,
                        score_threshold=None,
                        top_k=None
                    )


                    second_stage_superglue_outputs['keypoints0'][:, 0] += x_offset1
                    second_stage_superglue_outputs['keypoints0'][:, 1] += y_offset1
                    second_stage_superglue_outputs['keypoints1'][:, 0] += x_offset2
                    second_stage_superglue_outputs['keypoints1'][:, 1] += y_offset2

                    mkpts0 = np.concatenate([
                        superglue_outputs['keypoints0'], second_stage_superglue_outputs['keypoints0']
                    ], axis=0)
                    mkpts1 = np.concatenate([
                        superglue_outputs['keypoints1'],
                        second_stage_superglue_outputs['keypoints1']
                    ], axis=0)
                else:
                    mkpts0 = superglue_outputs['keypoints0']
                    mkpts1 = superglue_outputs['keypoints1']
            else:
                mkpts0 = sift_outputs['keypoints0']
                mkpts1 = sift_outputs['keypoints1']
            
            n_matches = len(mkpts1)
            group  = f_match.require_group(key1)
            if n_matches >= 5:
                group.create_dataset(key2, data=np.concatenate([mkpts0, mkpts1], axis=1))

    kpts = defaultdict(list)
    match_indexes = defaultdict(dict)
    total_kpts=defaultdict(int)
    with h5py.File(f'{feature_dir}/matches_loftr.h5', mode='r') as f_match:
        for k1 in f_match.keys():
            group  = f_match[k1]
            for k2 in group.keys():
                matches = group[k2][...]
                total_kpts[k1]
                kpts[k1].append(matches[:, :2])
                kpts[k2].append(matches[:, 2:])
                current_match = torch.arange(len(matches)).reshape(-1, 1).repeat(1, 2)
                current_match[:, 0]+=total_kpts[k1]
                current_match[:, 1]+=total_kpts[k2]
                total_kpts[k1]+=len(matches)
                total_kpts[k2]+=len(matches)
                match_indexes[k1][k2]=current_match

    for k in kpts.keys():
        kpts[k] = np.round(np.concatenate(kpts[k], axis=0))
    unique_kpts = {}
    unique_match_idxs = {}
    out_match = defaultdict(dict)
    for k in kpts.keys():
        uniq_kps, uniq_reverse_idxs = torch.unique(torch.from_numpy(kpts[k]),dim=0, return_inverse=True)
        unique_match_idxs[k] = uniq_reverse_idxs
        unique_kpts[k] = uniq_kps.numpy()
    for k1, group in match_indexes.items():
        for k2, m in group.items():
            m2 = deepcopy(m)
            m2[:,0] = unique_match_idxs[k1][m2[:,0]]
            m2[:,1] = unique_match_idxs[k2][m2[:,1]]
            mkpts = np.concatenate([unique_kpts[k1][ m2[:,0]],
                                    unique_kpts[k2][  m2[:,1]],
                                   ],
                                   axis=1)
            unique_idxs_current = get_unique_idxs(torch.from_numpy(mkpts), dim=0)
            m2_semiclean = m2[unique_idxs_current]
            unique_idxs_current1 = get_unique_idxs(m2_semiclean[:, 0], dim=0)
            m2_semiclean = m2_semiclean[unique_idxs_current1]
            unique_idxs_current2 = get_unique_idxs(m2_semiclean[:, 1], dim=0)
            m2_semiclean2 = m2_semiclean[unique_idxs_current2]
            out_match[k1][k2] = m2_semiclean2.numpy()
    with h5py.File(f'{feature_dir}/keypoints.h5', mode='w') as f_kp:
        for k, kpts1 in unique_kpts.items():
            f_kp[k] = kpts1
    
    with h5py.File(f'{feature_dir}/matches.h5', mode='w') as f_match:
        for k1, gr in out_match.items():
            group  = f_match.require_group(k1)
            for k2, match in gr.items():
                group[k2] = match
    return

def import_into_colmap(img_dir, feature_dir ='.featureout', database_path = 'colmap.db'):
    
    db = COLMAPDatabase.connect(database_path)
    db.create_tables()
    single_camera = False
    fname_to_id = add_keypoints(db, feature_dir, img_dir, 'simple-radial', single_camera)
    add_matches(
        db,
        feature_dir,
        fname_to_id,
    )

    db.commit()
    return

In [ ]:
validation_scenes = {
    #'haiper': ['bike', 'chairs', 'fountain'],
    'heritage': ['cyprus'],
    #'urban': ['kyiv-puppet-theater']
}

for dataset in validation_scenes.keys():
    
    dataset_directory = competition_dataset / 'train' / dataset
    
    for scene in validation_scenes[dataset]:
        
        df_scene = df.loc[df['scene'] == scene]
        
        scene_mean_memory_usage = df_scene['memory_usage'].mean()
        if scene_mean_memory_usage <= 8:
            lru_cache_max_size = 32
        else:
            lru_cache_max_size = 2
            
        @lru_cache(maxsize=lru_cache_max_size)
        def read_image(image_path):
            return cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
        
        scene_directory = dataset_directory / scene
        scene_image_directory = scene_directory / 'images'
        image_paths = sorted(glob(str(scene_image_directory / '*')))
        scene_image_count = len(image_paths)
        
        # Select images if scene image count is above the specified threshold
        max_scene_image_count = 200
        if scene_image_count > max_scene_image_count:
            # Select images with a step size
            step_size = int(np.ceil(len(image_paths) / max_scene_image_count))
            image_paths = image_paths[::step_size]
            
        feature_dir = f'featureout/{dataset}_{scene}'
        if not os.path.isdir(feature_dir):
            os.makedirs(feature_dir, exist_ok=True)
            
        scene_image_pair_indices = create_image_pairs(image_paths)
        
        print(
            f'''
            Dataset: {dataset} - Scene: {scene}
            Image count: {scene_image_count} - Pair count: {len(scene_image_pair_indices)}
            Average memory usage per image: {scene_mean_memory_usage:.4f} - LRU cache max size: {lru_cache_max_size}
            '''
        )
        
        match(
            read_image,
            image_paths,
            scene_image_pair_indices,
            feature_dir=feature_dir,
            loftr_model=loftr_model,
            superglue_model=superglue_model
        )
        
        database_path = f'{feature_dir}/colmap.db'
        if os.path.isfile(database_path):
            os.remove(database_path)

        import_into_colmap(str(scene_directory / 'images'), feature_dir=feature_dir,database_path=database_path)
        output_path = f'{feature_dir}/colmap_rec'

        pycolmap.match_exhaustive(database_path)

        mapper_options = pycolmap.IncrementalMapperOptions()
        mapper_options.min_model_size = 3
        os.makedirs(output_path, exist_ok=True)
        reconstructions = pycolmap.incremental_mapping(
            database_path=database_path,
            image_path=str(scene_directory / 'images'),
            output_path=output_path,
            options=mapper_options
        )
    
        if len(reconstructions) > 0:

            best_registered_image_count = 0
            best_reconstruction_idx = None

            for reconstruction_idx in reconstructions.keys():
                if reconstructions[reconstruction_idx].num_reg_images() > best_registered_image_count:
                    best_reconstruction_idx = reconstruction_idx
                    best_registered_image_count = reconstructions[reconstruction_idx].num_reg_images()

            best_reconstruction = reconstructions[best_reconstruction_idx]
        else:
            best_registered_image_count = 0
            best_reconstruction_idx = None
            best_reconstruction = None

        print(
            f'''
            Dataset: {dataset} - Scene: {scene}
            Reconstruction count: {len(reconstructions)}
            Best reconstruction registered image count: {best_registered_image_count}/{len(image_paths)}
            '''
        )

        if best_reconstruction is not None:
            registered_images = {image.name: image for image in best_reconstruction.images.values()}
        else:
            registered_images = {}

        for idx, row in df.loc[df['scene'] == scene].iterrows():
            if row['image_id'] in registered_images:
                rotation_matrix_prediction = deepcopy(registered_images[row['image_id']].rotmat())
                translation_vector_prediction = deepcopy(registered_images[row['image_id']].tvec)
                df.loc[idx, 'rotation_matrix_prediction'] = ';'.join([str(x) for x in rotation_matrix_prediction.reshape(-1)])
                df.loc[idx, 'translation_vector_prediction'] = ';'.join([str(x) for x in translation_vector_prediction.reshape(-1)])
            else:
                df.loc[idx, 'rotation_matrix_prediction'] = np.nan
                df.loc[idx, 'translation_vector_prediction'] = np.nan

        # Fill unregistered images rotation matrices with the prediction mean or zeros
        scene_rotation_matrix_predictions = df.loc[df['scene'] == scene, 'rotation_matrix_prediction'].dropna().apply(lambda x: np.array(str(x).split(';'), dtype=np.float64).reshape(1, 3, 3)).values
        if scene_rotation_matrix_predictions.shape[0] == 0:
            rotation_matrix_fill_value = np.zeros((3, 3))
        else:
            rotation_matrix_fill_value = np.mean(np.concatenate(scene_rotation_matrix_predictions, axis=0), axis=0)
        df.loc[(df['scene'] == scene) & (df['rotation_matrix_prediction'].isnull()), 'rotation_matrix_prediction'] = ';'.join([str(x) for x in rotation_matrix_fill_value.reshape(-1)])

        # Fill unregistered images translation vectors with the prediction mean or zeros
        scene_translation_vector_predictions = df.loc[df['scene'] == scene, 'translation_vector_prediction'].dropna().apply(lambda x: np.array(str(x).split(';'), dtype=np.float64).reshape(1, 3)).values
        if scene_translation_vector_predictions.shape[0] == 0:
            translation_vector_fill_value = np.zeros((3, 1))
        else:
            translation_vector_fill_value = np.mean(np.concatenate(scene_translation_vector_predictions, axis=0), axis=0)
        df.loc[(df['scene'] == scene) & (df['translation_vector_prediction'].isnull()), 'translation_vector_prediction'] = ';'.join([str(x) for x in translation_vector_fill_value.reshape(-1)])


In [ ]:
evaluate(df.dropna(subset=['rotation_matrix_prediction', 'translation_vector_prediction']))